<h1>DSM100
Artificial Intelligence
Jupyter Notebook</h1>


<h3>1.Overall Structure</h3>

<h5>1.1 Main entities</h5>
strange weather -> rainy, cloudy -> playable

<h5>1.2 Strange weather </h5>
P(strange weather) = 0.6

<h5>1.3 Rainy </h5>
P(Rainy | strange weather) = 0.6<br>
P(Rainy | not strange weather) = 0.1<br>
P(not Rainy | strange weather) = 0.4<br>
P(not Rainy | not strange weather) = 0.9<br>

<h5>1.4 Cloudy </h5>
P(Cloudy | strange weather) = 0.9<br>
P(Cloudy | not strange weather) = 0.3<br>
P(not Cloudy | strange weather) = 0.1<br>
P(not Cloudy | not strange weather) = 0.7<br>


<h5>1.5 Playable </h5>
P(Playbable | Yes Rainy, Yes Cloudy) = 0.01<br>
P(Playbable | Yes Rainy, No Cloudy) = 0.2<br>
P(Playbable | No Rainy, Yes Cloudy) = 0.8<br>
P(Playbable | No Rainy, No Cloudy) = 0.99<br>

<h3>2. Implementation</h3>

<h5>2.1 Import requirements</h5>

Importing required methods from pgmpy. 

In [3]:
import numpy as np


In [4]:
from pgmpy.factors.discrete import TabularCPD

In [5]:
from pgmpy.models import BayesianModel

<h5>2.2 Initiating Model</h5> 

Defining model structure. Here a list of edges is passed. 

In [6]:
tennis_model = BayesianModel([('Strange Weather', 'Rainy'),
                              ('Strange Weather', 'Cloudy'),
                              ('Rainy', 'Playable'),
                              ('Cloudy', 'Playable')])

C:\Users\KK\anaconda3\lib\site-packages\pgmpy\models\BayesianModel.py:10: FutureWarning: BayesianModel has been renamed to BayesianNetwork. Please use BayesianNewtork class, BayesianModel will be removed in future.
  FutureWarning,


<h5>2.3 CPD's</h5>

Defining all conditional probabilites tables from the graphical representation show in the report or in the overview at the beginning of the notebook. The CPD's are fromed by a method called TabularCPD.

In [7]:
strangeweather_cpd = TabularCPD(
    variable = 'Strange Weather',
    variable_card = 2,   # cardinality
    values = [[.6], [.4]])   # ['yes', 'no']

In [8]:
rainy_cpd = TabularCPD(
    variable = 'Rainy',
    variable_card = 2,
    values = [[.7, .1],
              [.3, .9]],
    evidence = ['Strange Weather'],
    evidence_card = [2])

In [9]:
cloudy_cpd = TabularCPD(
    variable = 'Cloudy',
    variable_card = 2,
    values = [[.9, .3],
              [.1, .7]],
    evidence = ['Strange Weather'],
    evidence_card = [2])

In [10]:
playable_cpd = TabularCPD(
    variable = 'Playable',
    variable_card = 2,
    values = [[0.01, .20, .80, 0.99],
              [0.99, .80, .20, 0.01]],
    evidence = ['Rainy', 'Cloudy'],
    evidence_card = [2,2])

<h5>2.4 Visualize the CPD</h5>

Printing the "playable" conditional probability distribution by calling the method get_cpds and printing. 

In [11]:
tennis_model.add_cpds(strangeweather_cpd, rainy_cpd, cloudy_cpd, playable_cpd)

In [12]:
tennis_model.get_cpds()

[<TabularCPD representing P(Strange Weather:2) at 0x2119cccd3c8>,
 <TabularCPD representing P(Rainy:2 | Strange Weather:2) at 0x2119ccd00c8>,
 <TabularCPD representing P(Cloudy:2 | Strange Weather:2) at 0x2119ccd0748>,
 <TabularCPD representing P(Playable:2 | Rainy:2, Cloudy:2) at 0x2119ccd3648>]

In [43]:
# Printing a CPD with it's state names defined.
print(tennis_model.get_cpds('Playable'))

+-------------+-----------+-----------+-----------+-----------+
| Rainy       | Rainy(0)  | Rainy(0)  | Rainy(1)  | Rainy(1)  |
+-------------+-----------+-----------+-----------+-----------+
| Cloudy      | Cloudy(0) | Cloudy(1) | Cloudy(0) | Cloudy(1) |
+-------------+-----------+-----------+-----------+-----------+
| Playable(0) | 0.01      | 0.2       | 0.8       | 0.99      |
+-------------+-----------+-----------+-----------+-----------+
| Playable(1) | 0.99      | 0.8       | 0.2       | 0.01      |
+-------------+-----------+-----------+-----------+-----------+


<h5>2.5 Active Trail and Local Independencies</h5>

Firstly, we find an active trail. That means that for any two variables A and B in the network, if a change in A influences values B, then there is an active trail. Here one can see which nodes are affected by "Strange Weather". 

Secondly, we find the independencies given in the network. Here local independencies show a variable which is independent of its non-descendents given its parents. Thus showing here for example that "playable" is affected by "strange weather", which is affected by "rainy" and "cloudy". 

In [13]:
tennis_model.active_trail_nodes('Strange Weather')

{'Strange Weather': {'Cloudy', 'Playable', 'Rainy', 'Strange Weather'}}

In [14]:
tennis_model.local_independencies('Strange Weather')

In [15]:
tennis_model.get_independencies()

(Playable ⟂ Strange Weather | Rainy, Cloudy)
(Strange Weather ⟂ Playable | Rainy, Cloudy)
(Rainy ⟂ Cloudy | Strange Weather)
(Cloudy ⟂ Rainy | Strange Weather)

<h5>2.6 Inference from the model</h5>

Now comes the interesting part. Here the prediction of values will take place with a basic method called "Variable Elimination". 

Then one can see the probabilities. For example the probabilities for "playable" are 0.5012 and 0.4988 respectively. The same process is done for the variables "Rainy", "Cloudy" and "Strange Weather". 

Further down the conditional distribution will also be computed, there one must just pass an additional argument. For example variabel "Playable", evidence "Strange Weather", shows that that the probability is 0.2660 that the courts are playable if there is strange weather and 0.7340 respectively. 

In [16]:
from pgmpy.inference import VariableElimination

In [17]:
tennis_infer = VariableElimination(tennis_model)

<h6>2.6.1 Inference "Playable"</h6>

In [18]:
prob_playable = tennis_infer.query(variables = ['Playable'], joint=False)

In [19]:
print(prob_playable['Playable'])

+-------------+-----------------+
| Playable    |   phi(Playable) |
+=============+=================+
| Playable(0) |          0.5012 |
+-------------+-----------------+
| Playable(1) |          0.4988 |
+-------------+-----------------+


<h6>2.6.2 Inference "Rainy"</h6>

In [20]:
prob_rainy = tennis_infer.query(variables = ['Rainy'], joint=False)

In [21]:
print(prob_rainy['Rainy'])

+----------+--------------+
| Rainy    |   phi(Rainy) |
+==========+==============+
| Rainy(0) |       0.4600 |
+----------+--------------+
| Rainy(1) |       0.5400 |
+----------+--------------+


<h6>2.6.3 Inference "Cloudy"</h6>

In [22]:
prob_cloudy = tennis_infer.query(variables = ['Cloudy'], joint=False)

In [23]:
print(prob_cloudy['Cloudy'])

+-----------+---------------+
| Cloudy    |   phi(Cloudy) |
+===========+===============+
| Cloudy(0) |        0.6600 |
+-----------+---------------+
| Cloudy(1) |        0.3400 |
+-----------+---------------+


<h6>2.6.4 Inference "Strange Weather"</h6>

In [24]:
prob_strangeweather = tennis_infer.query(variables = ['Strange Weather'], joint=False)

In [25]:
print(prob_strangeweather['Strange Weather'])

+--------------------+------------------------+
| Strange Weather    |   phi(Strange Weather) |
+====================+========================+
| Strange Weather(0) |                 0.6000 |
+--------------------+------------------------+
| Strange Weather(1) |                 0.4000 |
+--------------------+------------------------+


<h6>2.6.5 Conditional Distribution "Playable", "Strange Weather"</h6>

In [26]:
prob_playable_strangeweather = tennis_infer.query(
        variables = ['Playable'], joint=False,
        evidence = {'Strange Weather':0})

In [27]:
print(prob_playable_strangeweather['Playable'])

+-------------+-----------------+
| Playable    |   phi(Playable) |
+=============+=================+
| Playable(0) |          0.2660 |
+-------------+-----------------+
| Playable(1) |          0.7340 |
+-------------+-----------------+


<h6>2.6.6 Conditional Distribution "Rainy", "Strange Weather"</h6>

In [47]:
prob_rainy_strangeweather = tennis_infer.query(
        variables = ['Rainy'], joint=False,
        evidence = {'Strange Weather':0})

In [48]:
print(prob_rainy_strangeweather['Rainy'])

+----------+--------------+
| Rainy    |   phi(Rainy) |
+==========+==============+
| Rainy(0) |       0.7000 |
+----------+--------------+
| Rainy(1) |       0.3000 |
+----------+--------------+


<h6>2.6.7 Conditional Distribution "Cloudy", "Strange Weather"</h6>

In [30]:
prob_cloudy_strangeweather = tennis_infer.query(
        variables = ['Cloudy'], joint=False,
        evidence = {'Strange Weather':0})

In [31]:
print(prob_cloudy_strangeweather['Cloudy'])

+-----------+---------------+
| Cloudy    |   phi(Cloudy) |
+===========+===============+
| Cloudy(0) |        0.9000 |
+-----------+---------------+
| Cloudy(1) |        0.1000 |
+-----------+---------------+


<h6>2.6.8 Map Query</h6>

Finally, the most probable state rather than the probability distribution must be found out. For that we will use two variable as evidence. For example the most probable state for "playable" is that it is not playable. This is done with a method called map_query. Here it shows that if there is strange weather and it is rainy, then most probably it is not playable. Same goes for cloudy.

In [53]:
prob_strangeweather_playable_rainy = tennis_infer.map_query(
        variables = ['Playable'],
        evidence = {'Strange Weather':0, 'Rainy':0})

In [54]:
print(prob_strangeweather_playable_rainy['Playable'])

1


In [58]:
prob_playable_strangeweather_cloudy = tennis_infer.map_query(
        variables = ['Playable'],
        evidence = {'Strange Weather':0, 'Cloudy':0})

In [59]:
print(prob_playable_strangeweather_cloudy['Playable'])

1
